#**DATA PREPROCESSING**

**Input: raw data - Output: transactions**

In [ ]:
# First, we loaded the e-commerce dataset to google.colab from drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks')
!pwd

/content/gdrive/My Drive/Colab Notebooks


In [ ]:
!ls

'Copy of 100-pandas-puzzles.ipynb'
'Copy of DangNgocMinhHuy_ITITIU18052_lab6-EDA-ML-CC.ipynb'
'Copy of Data-structures-Lists_Tuples.ipynb'
'Copy of lab1-pandas- Indexing and selecting data.ipynb'
'DangNgocMinhHuy_ITITIU18052_Lab5_AI (1).ipynb'
 DangNgocMinhHuy_ITITIU18052_Lab5_AI.ipynb
 DangNgocMinhHuy_ITITIU18052_Lab5_submit.ipynb
'DangNgocMinhHuy_ITITIU18052_lab6_EDA_ML_CC (1).ipynb'
 DangNgocMinhHuy_ITITIU18052_lab6_EDA_ML_CC.ipynb
 DangNgocMinhHuy_ITITIU18052_lab6-EDA-ML-CC.ipynb
'DangNgocMinhHuy_ITITIU18052_PPL_Lab02 (1).ipynb'
 DangNgocMinhHuy_ITITIU18052_PPL_Lab02.ipynb
 DangNgocMinhHuy_ITITIU18052_PPL_Lab03.ipynb
'DangNgocMinhHuy_ITITIU18052_PPL_Lab1 (1).ipynb'
 DangNgocMinhHuy_ITITIU18052_PPL_Lab1.ipynb
 DangNgocMinhHuy_ITITIU18052_PPL_Lab4.ipynb
 DangNgocMinhHuy_ITITIU18052_PPL_Lab6.ipynb
 DangNgocMinhHuy_ITITIU18052_submit_Excercise_KNN_PredictDiabetes.ipynb
 data.csv
'Excercise_KNN_PredictDiabetes (1).ipynb'
'Excercise_KNN_PredictDiabetes (2).ipynb'
'Excercise_KNN_PredictDi

**1. Data Cleaning**

In [ ]:
# modules we'll use
import pandas as pd
import numpy as np

# read in all our data
ecommerce_data = pd.read_csv('data.csv', engine='python', parse_dates=['InvoiceDate'])

# set seed for reproducibility
np.random.seed(0)

ParserError: ignored

In [ ]:
ecommerce_data.head()

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# get the number of missing data points per column
missing_values_count = ecommerce_data.isnull().sum()
missing_values_count[:]

# total missing values
total_cells = np.product(ecommerce_data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
percent_missing

In [ ]:
ecommerce_data["InvoiceDate"] = pd.to_datetime(ecommerce_data.InvoiceDate, cache=True)

ecommerce_data.InvoiceDate.max() - ecommerce_data.InvoiceDate.min()

In [ ]:
ecommerce_data["Revenue"] = ecommerce_data.Quantity * ecommerce_data.UnitPrice

ecommerce_data["Year"] = ecommerce_data.InvoiceDate.dt.year
ecommerce_data["Quarter"] = ecommerce_data.InvoiceDate.dt.quarter
ecommerce_data["Month"] = ecommerce_data.InvoiceDate.dt.week
ecommerce_data["Week"] = ecommerce_data.InvoiceDate.dt.week
ecommerce_data["Weekday"] = ecommerce_data.InvoiceDate.dt.weekday
ecommerce_data["Day"] = ecommerce_data.InvoiceDate.dt.day
ecommerce_data["Dayofyear"] = ecommerce_data.InvoiceDate.dt.dayofyear
#ecommerce_data['Date'] = pd.to_datetime(ecommerce_data[['Year', 'Month', 'Day']])
#ecommerce_data["Date"] = pd.to_datetime(ecommerce_data[['Year', 'Month', 'Day']])

In [ ]:
from scipy import stats
from mlxtend.preprocessing import minmax_scaling
import seaborn as sns
import matplotlib.pyplot as plt

np.random.seed(0)

In [ ]:
df = ecommerce_data.copy()
df.describe()

In [ ]:
fig,ax = plt.subplots(nrows=2,figsize=(20,7))
sns.boxplot(df[(df['InvoiceNo'].str[0]=='c')|(df['InvoiceNo'].str[0]=='C')]['Quantity'],ax=ax[0])
sns.boxplot(df[(df['InvoiceNo'].str[0]=='c')|(df['InvoiceNo'].str[0]=='C')]['UnitPrice'],ax=ax[1])
ax[0].title.set_text("Cancelled Orders 'C' quantity and price distribution")

In [ ]:
def check_hypothesis_cancelled_order(df):
  failed=0
  passed=0
  neg_qty=df[df["Quantity"]<0]
  pos_qty=df[df["Quantity"]>0]
  for ind in neg_qty.index:
    if(neg_qty['CustomerID'][ind]):
            p = pos_qty[
                (pos_qty['CustomerID'] == neg_qty['CustomerID'][ind])&
                (pos_qty['Quantity'] <= abs(neg_qty['Quantity'][ind]))&
                ((pos_qty['InvoiceDate'] - neg_qty['InvoiceDate'][ind]).dt.total_seconds()>=0)
            ]
            if(len(p)==0):
                failed+=1
            else:
                passed+=1
    if(failed>passed):
        print("Hypothesis Rejected!")
        print("Failed Counts:"+str(failed)+" Passed Counts:"+str(passed))
        print("Approximately "+str(int(failed/(failed + passed)*100)) + "% rows didn't satisfy the condition")

check_hypothesis_cancelled_order(df)

# Step 3: Implement FPGrowth, to build a prediction model based on the sequences in the (clustered) training datasets. 

In [15]:
!pip install pyspark

In [16]:
from pyspark.ml.fpm import FPGrowth
from pyspark.sql import SparkSession
from tqdm.autonotebook import tqdm

class FPGModel:
  """
  input:  datafarme
  output: prediction
  """
  def __init__(self):
      self.spark = SparkSession.builder.appName("FPGrowth_Model").getOrCreate()
      self.model = None
      self.itemsCol = None

  def fit(self, 
          data, 
          itemsCol, 
          mSup=0.5, 
          mConf=0.6):
      self.itemsCol = itemsCol
      fpGrowth = FPGrowth(itemsCol=itemsCol, minSupport=mSup, minConfidence=mConf)
      self.df = self.spark.createDataFrame(data)
      self.model = fpGrowth.fit(self.df)

  def show(self):
      # Display frequent itemsets.
      self.model.freqItemsets.show()

      # Display generated association rules.
      self.model.associationRules.show()

      # transform examines the input items against all the association rules and summarize the
      # consequents as prediction
      self.model.transform(self.df).show()
    
  def predict(self, data):
    data = pd.DataFrame([(data, )], columns=[self.itemsCol])
    data = self.spark.createDataFrame(data)
    return sorted(self.model.transform(data).first().prediction)

  def evaluate(self, data):
    score = 0
    total_data = len(data)
    for _, test in tqdm(enumerate(data)):
      if len(test) == 1:
        total_data -= 1
        continue
      total_test = len(test)
      idx = 0
      while idx != len(test)-1:
        if test[idx+1] not in self.predict(test[:idx+1]):
          break
        idx += 1
      score += idx/total_test
    return score/total_data
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
# !gdown https://drive.google.com/uc?id=1AkA7TBwSIV35AeU8OJQtf_LmgtR_fAWv
# !gdown https://drive.google.com/uc?id=1QzgOv75YS7kDKswXcKBf44oLW1_M3lRL
!gdown https://drive.google.com/uc?id=1npVa0d3ct_ztIqHZjLQCtuV_w86Cj62O
!gdown https://drive.google.com/uc?id=1vsRSzinZKbk137ZFHeIJAH-4-tVfHvRo

Downloading...
From: https://drive.google.com/uc?id=1npVa0d3ct_ztIqHZjLQCtuV_w86Cj62O
To: /content/Cluster_Data.csv
2.32MB [00:00, 75.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vsRSzinZKbk137ZFHeIJAH-4-tVfHvRo
To: /content/Cluster_Test_Data.csv
100% 235k/235k [00:00<00:00, 34.7MB/s]


In [18]:
import pandas as pd
import numpy as np

df = pd.read_csv("Cluster_Data.csv")
df = df.dropna()
cluster = {}

In [21]:
for idx in df.Cluster.unique():
  print(f'Training Cluster {idx}')
  data = df.loc[df.Cluster == idx]
  data = data.groupby('InvoiceNo')['ProductName'].apply(lambda x: list(set(x)))
  data = pd.DataFrame(data, columns=['ProductName'])
  model = FPGModel()
  model.fit(data = data, itemsCol = 'ProductName', mSup=0.001, mConf=0.001)
  cluster[idx] = model

Training Cluster 0.0
Training Cluster 3.0
Training Cluster 2.0
Training Cluster 1.0


In [30]:
df.loc[df.Cluster == 1]

,Cluster,InvoiceNo,ProductName
12,1.0,536370,3545
26,1.0,536378,1700
27,1.0,536378,3305
31,1.0,536378,1722
42,1.0,536381,979
...,...,...,...
147004,1.0,581585,1776
147005,1.0,581585,196
147006,1.0,581585,3216
147007,1.0,581586,990


In [22]:
test_df = pd.read_csv('Cluster_Test_Data.csv')
output = {}
for idx in test_df.Cluster.unique():
  print(f'Evaluating Cluster {idx}')
  data = test_df.loc[test_df.Cluster == idx]
  data = data.groupby('InvoiceNo')['ProductName'].apply(lambda x: list(set(x)))
  data = data.tolist()
  output[idx] = cluster[idx].evaluate(data)
  print(output[idx])

print(f'Final Evaluation: {output}')

Evaluating Cluster 0.0



0.016301471366571403
Evaluating Cluster 1.0



0.0
Evaluating Cluster 2.0



0.0
Evaluating Cluster 3.0



0.0
Final Evaluation: {0.0: 0.016301471366571403, 1.0: 0.0, 2.0: 0.0, 3.0: 0.0}
